<a href="https://colab.research.google.com/github/Auspiland/myToy/blob/main/%EA%B5%AC%EC%8A%AC%ED%83%88%EC%B6%9C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import numpy as np
import copy

a = input()
a = a.split()
M=int(a[0])
N=int(a[1])

table = []
copied_table = []
for i in range(M):
  tempList = list(a[i+2])
  table.append(tempList)
  copied_table.append(tempList)
table = np.array(table)
temp=np.where(table == "R")
R_point = (temp[0][0],temp[1][0])
temp=np.where(table == "B")
B_point = (temp[0][0],temp[1][0])
temp=np.where(table == "O")
O_point = (temp[0][0],temp[1][0])
stored = copy.deepcopy([copied_table,R_point,B_point,O_point])

# print(table)
# print(R_point, B_point, O_point)
# print("---------------------")

def tiltValue(point):
  x,y=point
  row=table[x]
  column=np.transpose(table)[y]
  valset = []
  for l1, l2, direction in (('left','right',row),('up','down',column)):
    way=l1
    dot_count =0
    desti = None
    switch = 0
    for k in direction:
      if k == 'R':
        if desti == True:
          switch=1
        else:
          valset.append((l1,dot_count))
        way=l2
        dot_count =0
      elif k == '.':
        dot_count +=1
      elif k == '#':
        if way == l2:
          valset.append((l2,dot_count))
          break
        dot_count =0
        if desti==True:
          desti==False
      elif k == 'O':
        if desti == False:
          valset.append((l2,0))
          break
        elif way == l2:
          valset.append((l2,1001))
          break
        desti = True
      elif k == 'B':
        if desti==True:
          valset.append((l1,0))
          switch=0
        desti = False
    if switch==1:
      valset.append((l1,1001))
  # print("Valset is :",valset)
  return valset

def tiltB(point,way):
  x,y=point
  dot_count=0
  if way=='left':
    line=table[x]
  elif way=='up':
    line=np.transpose(table)[y]
  elif way=='right':
    line=np.flip(table[x])
  elif way=='down':
    line=np.flip(np.transpose(table)[y])
  else:
    raise ValueError("Unexpected way ;",way)
  for k in line:
    if k == '.':
      dot_count += 1
    elif k == '#':
      dot_count = 0
    elif k == 'B':
      return dot_count
  raise ValueError("Unexpected line or something ;",line,dot_count)

def tilt(rp,bp,rv,bv,way):
  x1,y1=rp
  x2,y2=bp
  if way=='left':
    table[x1][y1-rv] = 'R'
    table[x1][y1] = '.'
    table[x2][y2-bv] = 'B'
    if table[x2][y2] == 'B'and bv!=0:
      table[x2][y2] = '.'
    return (x1,y1-rv),(x2,y2-bv)
  elif way=='up':
    table[x1-rv][y1] = 'R'
    table[x1][y1] = '.'
    table[x2-bv][y2] = 'B'
    if table[x2][y2] == 'B' and bv!=0:
      table[x2][y2] = '.'
    return (x1-rv,y1),(x2-bv,y2)
  elif way=='right':
    table[x1][y1+rv] = 'R'
    table[x1][y1] = '.'
    table[x2][y2+bv] = 'B'
    if table[x2][y2] == 'B'and bv!=0:
      table[x2][y2] = '.'
    return (x1,y1+rv),(x2,y2+bv)
  elif way=='down':
    table[x1+rv][y1] = 'R'
    table[x1][y1] = '.'
    table[x2+bv][y2] = 'B'
    if table[x2][y2] == 'B'and bv!=0:
      table[x2][y2] = '.'
    return (x1+rv,y1),(x2+bv,y2)


answer =[]
n=0
n_count=1000
while n_count > 0:
  if n_count ==1000:
    n_count =0
  prev=None
  # print("-------------------{}th round & n_count:{} / bin(n):{}------------------------".format(n+1,n_count,bin(n)))
  if n>16:
    print("----------------------------n is over16: ",n)
    break
  for m in range(10):
    temp = np.transpose(tiltValue(R_point))
    temp1 = np.array([int(num) for num in temp[1]])
    temp = temp[0]
    if max(temp1) >1000:
      answer.append(m+1)
      break
    elif m == 9:
      answer.append(20)
      break
    if prev != None:
      if prev == 'left':
        prevp = 'right'
      elif prev == 'right':
        prevp = 'left'
      elif prev == 'up':
        prevp = 'down'
      elif prev == 'down':
        prevp = 'up'
      i = np.where(temp==prevp)[0][0]
      temp=np.delete(temp,i)
      temp1=np.delete(temp1,i)
    temp2=np.array([int(bool(ii)) for ii in temp1])
    if max(temp2) == 0:
      answer.append(20)
      break
    temp_MI = np.where(temp2==max(temp2))[0]
    if len(temp_MI) ==1:
      maxIndex = temp_MI[0]
    elif len(temp_MI)==2:
      if n==0:
        n_count+=1
      n_list = list(bin(n))
      n_list.pop(0)
      n_list.pop(0)
      if len(n_list)<n_count:
        n_list.insert(0,0)
      elif len(n_list)>n_count:
        n_count=-10
        break
      maxIndex = temp_MI[int(n_list[-n_count])]
    else:
      raise ValueError('Too much available ways ;',temp_MI)
    prev = temp[maxIndex]
    R_point,B_point=tilt(R_point,B_point,temp1[maxIndex],tiltB(B_point,prev),prev)
    # print(table)
  temp_copied_table,R_point,B_point,O_point = stored
  table=[]
  for ttt in temp_copied_table:
    table.append(copy.deepcopy(ttt))
  table = np.array(table)
  n+=1


# print("The FINAL answer set is : ",answer)
if min(answer) == 20:
  print(-1)
else:
  print(min(answer))


3 10 ########## #.O....RB# ##########
(1, 7) (1, 8) (1, 2)
-1


<<폐기된 코드>>

In [ ]:


  # for k in [row[y-1], row[y+1], column[x+1], column[x-1]]:
  #   if k == '#':
  #     valset.append(0)
  #   elif k == 'B' or valset[i] == 'R':
  #     [row[y-2], row[y+2], column[x+2], column[x-2]][len(valset)]
  #   elif k == 'O':
  #     if C == 'R':
  #       valset.append(5)
  #     else:
  #       valset.append(0)
  #   elif k == '.':
  #     valset.append(1)

def tiltUp(rp,bp,v=1):
  print("Up")
  value = 1
  for C,x,y in [rp,bp]:
    for i in range(1,N):
      if table[x-i][y]=='#' or table[x-i][y]=='B'or table[x-i][y]=='R':
        if i == 1:
          if C=='R':
            value*=0.5
        else:
          table[x-i+1][y] = C
          table[x][y] = '.'
          if C=='R':
            rp=('R',x-i+1,y)
          else:
            bp=('B',x-i+1,y)
          value*=1
          print(x,y," to ",x-i+1,y)
          print(table)
        if table[x-i][y]=='B':
          value *= -1
        break
      elif table[x-i][y]=='O':
        if C=='R':
          value*=5
        elif C=='B':
          return rp,bp,-1
        break
  if value < 0:
    print("codeBBBBB")
    return tiltUp(rp,bp,value)
  else:
    if v ==0 and value == 0:
      return rp,bp,0
    else:
      return rp,bp,v*value

def tiltDown(rp,bp,v=1):
  print("Down")
  value = 1
  for C,x,y in [rp,bp]:
    for i in range(1,N):
      if table[x+i][y]=='#' or table[x+i][y]=='B'or table[x+i][y]=='R':
        if i == 1:
          if C=='R':
            value*=0.5
        else:
          table[x+i-1][y] = C
          table[x][y] = '.'
          if C=='R':
            rp=('R',x+i-1,y)
          else:
            bp=('B',x+i-1,y)
          value*=1
          print(x,y," to ",x+i-1,y)
          print(table)
        if table[x+i][y]=='B':
          value *= -1
        break
      elif table[x+i][y]=='O':
        if C=='R':
          value*=5
        elif C=='B':
          return rp,bp,-1
        break
  if value < 0:
    print("codeBBBBB")
    return tiltUp(rp,bp,value)
  else:
    if v ==0 and value == 0:
      return rp,bp,0
    else:
      return rp,bp,v*value

def tiltLeft(rp,bp,v=1):
  print("Left")
  value = 1
  for C,x,y in [rp,bp]:
    for i in range(1,M):
      if table[x][y-i]=='#' or table[x][y-i]=='B'or table[x][y-i]=='R':
        if i == 1:
          if C=='R':
            value*=0.5
        else:
          table[x][y-i+1] = C
          table[x][y] = '.'
          if C=='R':
            rp=('R',x,y-i)
          else:
            bp=('B',x,y-i)
          value*=1
          print(x,y," to ",x,y-i+1)
          print(table)
        if table[x][y-i]=='B':
          value *= -1
        break
      elif table[x][y-i]=='O':
        if C=='R':
          value*=5
        elif C=='B':
          return rp,bp,-1
        break
  if value < 0:
    print("codeBBBBB")
    return tiltUp(rp,bp,value)
  else:
    if v ==0 and value == 0:
      return rp,bp,0
    else:
      return rp,bp,v*value

def tiltRight(rp,bp,v=1):
  print("Right")
  value = 1
  for C,x,y in [rp,bp]:
    for i in range(1,M):
      if table[x][y+i]=='#' or table[x][y+i]=='B' or table[x][y+i]=='R':
        if i == 1:
          if C=='R':
            value*=0.5
        else:
          table[x][y+i-1] = C
          table[x][y] = '.'
          if C=='R':
            rp=('R',x,y+i)
          else:
            bp=('B',x,y+i)
          value*=1
          print(x,y," to ",x,y+i-1)
          print(table)
        if table[x][y+i]=='B':
          print("codeBBBBB")
          value *= -1
        break
      elif table[x][y+i]=='O':
        if C=='R':
          value*=5
        elif C=='B':
          return rp,bp,-1
        break
  if value < 0:
    return tiltUp(rp,bp,value)
  else:
    if v ==0 and value == 0:
      return rp,bp,0
    else:
      return rp,bp,v*value


# R_point, B_point,v= tiltUp(R_point, B_point)
# print()
# print(R_point, B_point,v)
# print()
# R_point, B_point,v= tiltDown(R_point, B_point)
# print()
# print(R_point, B_point,v)
# print()
# R_point, B_point,v= tiltLeft(R_point, B_point)
# print()
# print(R_point, B_point,v)
# print()
# R_point, B_point,v= tiltRight(R_point, B_point)
# print()
# print(R_point, B_point,v)
# print()

# funcs=[tiltUp,tiltDown,tiltLeft,tiltRight]
# funcode = -1
# for i in range(10):
#   vals=[]
#   for k in range(4):
#     if k == funcode:
#       continue
#     vals.append(funcs[k](R_point, B_point)[-1])
#   if max(vals)>2:
#     print("The fucking number is : ",i+1)
#     break
#   funcode = vals.index(max(vals))
#   R_point, B_point,v = funcs[funcode](R_point, B_point)
# print('--------------------------------------')
# print(table)


[3, 4]